## Indian Start-Up Funding Ecosystem

In [ ]:
# Import requisite libraries
import pyodbc    
from dotenv import dotenv_values
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Collection

In [ ]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [ ]:
# Establish a connection to the DAPS Project database using the PyODBC library
connection = pyodbc.connect(connection_string)

In [ ]:
# Access the 2018 dataset
url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_LP1-Data_Analysis/main/startup_funding2018.csv'

dat1 = pd.read_csv(url)
dat1

In [ ]:
dat1.info()

In [ ]:
dat1["Amount"]

In [ ]:
dat1["Amount"].unique()

In [ ]:
dat1[dat1["Amount"].str.startswith("$")]

In [ ]:
# Assuming your dataframe is named dat1
dat1['Amount'] = dat1['Amount'].str.replace(',', '')
dat1

In [ ]:
# Define a function to convert rupees to dollars
def convert_to_dollars(Amount):
    if Amount.startswith('₹'):
        return float(Amount[1:]) * 0.0146
    else:
        return Amount
 
# Apply the conversion function to the 'amount' column
dat1['Amount'] = dat1['Amount'].apply(convert_to_dollars)

In [ ]:
dat1

In [ ]:
#dat1[dat1["Amount"].str.startswith("₹")]#

In [ ]:
#dat1[~dat1['Amount'].str.startswith(('₹', '$'))]#

In [ ]:
# Access the 2019 dataset

dat2 = pd.read_csv(r'C:\Users\HP SPECTRE\OneDrive\Desktop\startup_funding2019.csv')
dat2

In [ ]:
# Define a function to convert rupees to dollars
def convert_to_dollars(Amount):
    if Amount.startswith('₹'):
        return float(Amount[1:]) * 0.0146
    else:
        return Amount
 
# Apply the conversion function to the 'amount' column
dat2['Amount($)'] = dat2['Amount($)'].apply(convert_to_dollars)

In [ ]:
dat2

In [ ]:
#Renaming amount column in dataset 2#

dat2 = dat2.rename(columns={'Amount($)':'Amount'})
dat2

In [ ]:
dat2[~dat2['Amount'].str.startswith(('$'))]

In [ ]:
dat2['Amount'].unique()

In [ ]:
dat2_undisclosed = dat2[dat2['Amount']=='Undisclosed']
dat2_undisclosed

In [ ]:
dat2 = dat2[dat2['Amount']!='Undisclosed']
dat2['Amount'].unique()

In [ ]:
# Assuming your dataframe is named dat2
dat2['Amount'] = dat2['Amount'].str.startswith('$','')
dat2

In [ ]:
dat2

In [ ]:
dat1.to_csv('./dataset/startup_funding2018.csv')

In [ ]:
dat2.to_csv('./dataset/startup_funding2019.csv')

In [ ]:
dat3.to_csv('./dataset/startup_funding2020.csv')

In [ ]:
dat4.to_csv('./dataset/startup_funding2021.csv')

In [ ]:
query = "Select * from LP1_startup_funding2020"

dat3 = pd.read_sql(query, connection)
dat3

In [ ]:
query = "Select * from dbo.LP1_startup_funding2021"

dat4 = pd.read_sql(query, connection)
dat4

### Loading the datasets

In [ ]:
# Creating dataframes based on each dataset

data1 = 'startup_funding2018.csv'
data2 = 'startup_funding2019.csv'
data3 = 'startup_funding2020.csv'
data4 = 'startup_funding2021.csv'

df1 = pd.read_csv(data1)
df2 = pd.read_csv(data2)
df3 = pd.read_csv(data3)
df4 = pd.read_csv(data4)

In [ ]:
# Adding a year column to each dataset so that rows are not lost should it become necessary to merge all datasets later on
year_1 = 2018
df1['Year'] = year_1

year_2 = 2019
df2['Year'] = year_2

year_3 = 2020
df3['Year'] = year_3

year_4 = 2021
df4['Year'] = year_4

# Convert the year column from int data type to year data type
df1['Year'] = pd.to_datetime(df1['Year'], format='%Y').dt.year

df2['Year'] = pd.to_datetime(df2['Year'], format='%Y').dt.year

df3['Year'] = pd.to_datetime(df3['Year'], format='%Y').dt.year

df4['Year'] = pd.to_datetime(df4['Year'], format='%Y').dt.year

### Data Cleaning

In [ ]:
# Preview of df1
df1.head()

In [ ]:
# Preview of df2
df2.head()

In [ ]:
# Preview of df3
df3.head()

In [ ]:
# Preview of df4
df4.head()

In [ ]:
# Information on df1
df1.info()

In [ ]:
# Rename columns in df1 to match columns with similar content in df2, df3 and df4
df1 = df1.rename(columns = {'Company Name' : 'Company_Brand', 'Round/Series' : 'Stage', 'About Company' : 'What_it_does', 'Amount' : 'Amount($)'})

In [ ]:
# Extract Headquarters information from df1's Location column
df1['Headquarters'] = df1['Location'].str.split(',').str[0]
df1['Headquarters']

In [ ]:
df1['Sector'] = df1['Industry'].str.split(',').str[0]
df1[df1['Sector'] == 'Agritech']

In [ ]:
# Convert Amount from object to float data type

# First, remove currency symbol ₹
df1['Amount($)'] = df1['Amount($)'].str.replace('₹', '')

# Remove the comma (,) symbol
df1['Amount($)'] = df1['Amount($)'].str.replace(',', '')

# Remove the hyphen (—) symbol
df1['Amount($)'] = df1['Amount($)'].str.replace('—', '')

# Replace all empty spaces with 0
df1['Amount($)'] = df1['Amount($)'].str.replace('', '0')

# Replace all strings longer than 9 character
max_length = 9
df1['Amount($)'] = df1['Amount($)'].apply(lambda x: '0' if len(x) > max_length else x)

# df1['Amount($)'] = df1['Amount($)'].str.replace('0000000$0000000100000004000000030000000100000004000000050000000', '0')

# df1['Amount($)'] = df1['Amount($)'].str.replace('000000000000000$000000000000000100000000000000040000000000000003000000000000000100000000000000040000000000000005000000000000000', '0')

# Remove all white spaces
df1['Amount($)'] = df1['Amount($)'].str.strip()

df1['Amount($)'] = df1['Amount($)'].astype(np.float64).round(2)

In [ ]:
df1.drop(columns = ['Industry', 'Location'])

In [ ]:
# Checking information on df1 after cleaning the data
df1.info()

In [ ]:
df2.Sector

In [ ]:
# Checking df2 information before data cleaning
df2.info()

In [ ]:
# Fill missing values in Founded column
df2['Founded'] = df2['Founded'].fillna(0)

In [ ]:
# Fill missing values in Founded column
df2['Founded'] = df2['Founded'].fillna(0)

In [ ]:
# Convert Founded column from float to int
df2['Founded'] = df2['Founded'].astype(int)

In [ ]:
# Rename Headquarter to Headquarters
df2 = df2.rename(columns = {'Company/Brand' : 'Company_Brand', 'What it does' : 'What_it_does', 'HeadQuarter' : 'Headquarters'})

In [ ]:
# Convert Amount column from object to float data type

# Remove currency symbol $
df2['Amount($)'] = df2['Amount($)'].str.replace('$', '')

# Remove the comma (,) symbol
df2['Amount($)'] = df2['Amount($)'].str.replace(',', '')

# Replace all strings longer than 9 character
max_length = 9
df2['Amount($)'] = df2['Amount($)'].apply(lambda x: '0' if len(x) > max_length else x)

# Remove all white spaces
df2['Amount($)'] = df2['Amount($)'].str.strip()

df2['Amount($)'] = df2['Amount($)'].astype(np.float64).round(2)

In [ ]:
df2.isna().sum()

In [ ]:
df2[['Headquarters', 'Sector', 'Founders', 'Stage']] = df2[['Headquarters', 'Sector', 'Founders', 'Stage']].fillna('')

In [ ]:
df2.isna().sum()

In [ ]:
# Checking df2 after data cleaning
df2.info()

In [ ]:
# df3 information before data cleaning
df3.info()

In [ ]:
# Rename Headquarter to Headquarters
df3 = df3.rename(columns = {'HeadQuarter' : 'Headquarters', 'Amount' : 'Amount($)'})

In [ ]:
df3['Founded'] = df3['Founded'].fillna('0')

In [ ]:
# Convert Founded column from float to int
df3['Founded'] = df3['Founded'].astype(int)

In [ ]:
# # Convert Amount($) column from object to float data type

# # Remove currency symbol $
# df3['Amount($)'] = df3['Amount($)'].str.replace('$', '')

# # Remove the comma (,) symbol
# df3['Amount($)'] = df3['Amount($)'].str.replace(',', '')

# # Replace all strings longer than 9 character
# max_length = 9
# df3['Amount($)'] = df3['Amount($)'].apply(lambda x: '0' if len(x) > max_length else x)

# # Remove all white spaces
# df3['Amount($)'] = df3['Amount($)'].str.strip()

# df3['Amount($)'] = df3['Amount($)'].astype(np.float64).round(2)
df3['Amount($)'].isna().sum()

In [ ]:
df3['Amount($)'] = df3['Amount($)'].fillna(0)

In [ ]:
df3['Stage'].describe()

In [ ]:
df3.isna().sum()

In [ ]:
df3['Stage'] = df3['Stage'].fillna('Unknown')
df3['Stage'].isna().sum()

In [ ]:
df3.tail(20)

In [ ]:
# df3 after renaming column
df3.info()

In [ ]:
# df4 information before data cleaning
df4.info()

In [ ]:
# Rename Headquarter to Headquarters
df4 = df4.rename(columns = {'HeadQuarter' : 'Headquarters', 'Amount' : 'Amount($)'})

In [ ]:
df4['Founded'] = df4['Founded'].fillna(0)

In [ ]:
# Convert Founded column from float to int
df4['Founded'] = df4['Founded'].astype(int)

In [ ]:
df4['Stage'] = df4['Stage'].fillna('')

In [ ]:
# Convert Amount($) column from object to float data type

# Remove currency symbol $
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')
df4['Stage'] = df4['Stage'].str.replace('$', '')
df4['Stage'] = df4['Stage'].str.replace(',', '')
df4['Investor'] = df4['Investor'].str.replace('$', '')
df4['Investor'] = df4['Investor'].str.replace(',', '')
df4['Investor'] = df4['Investor'].str.strip()


In [ ]:
# Define a function to check if a string represents an integer
def is_string_integer(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Apply the function to the 'Stage' column to create a boolean mask
condition = df4['Stage'].apply(is_string_integer)

# Swap values between 'Stage' and 'Amount($)' columns where condition is True
temp = df4.loc[condition, 'Stage'].copy()
df4.loc[condition, 'Stage'] = df4.loc[condition, 'Amount($)']
df4.loc[condition, 'Amount($)'] = temp



# df4[['Amount($)', 'Stage']]
df4

In [ ]:
df4.groupby(by = 'Amount($)', as_index = False).sum()

In [ ]:
df4['Amount($)'].isna().sum()

In [ ]:
df4['Amount($)'] = df4['Amount($)'].fillna('0')

In [ ]:
# Define the pattern you want to search for in the 'Amount($)' column
pattern = 'Series'  # Replace 'pattern' with your desired pattern

# Identify rows where the 'Amount($)' column contains the specified pattern
condition = df4['Amount($)'].str.contains(pattern)

# Move the values from 'Amount($)' to 'Stage' where the pattern is found
df4.loc[condition, 'Stage'] = df4.loc[condition, 'Amount($)']

# Replace the values in 'Amount($)' with NaN where the pattern is found
df4.loc[condition, 'Amount($)'] = pd.NA

df4.groupby(by = 'Amount($)', as_index = False).sum()

In [ ]:
df4['Amount($)'] = df4['Amount($)'].fillna('0')

In [ ]:
# Define the pattern you want to search for in the 'Amount($)' column
pattern = 'Seed'  # Replace 'pattern' with your desired pattern

# Identify rows where the 'Amount($)' column contains the specified pattern
condition = df4['Amount($)'].str.contains(pattern)

# Move the values from 'Amount($)' to 'Stage' where the pattern is found
df4.loc[condition, 'Stage'] = df4.loc[condition, 'Amount($)']

# Replace the values in 'Amount($)' with NaN where the pattern is found
df4.loc[condition, 'Amount($)'] = pd.NA

df4.groupby(by = 'Amount($)', as_index = False).sum()

In [ ]:
df4['Amount($)'] = df4['Amount($)'].fillna('0')

In [ ]:
# Define the pattern you want to search for in the 'Amount($)' column
pattern = 'Pre-s'  # Replace 'pattern' with your desired pattern

# Identify rows where the 'Amount($)' column contains the specified pattern
condition = df4['Amount($)'].str.contains(pattern)

# Move the values from 'Amount($)' to 'Stage' where the pattern is found
df4.loc[condition, 'Stage'] = df4.loc[condition, 'Amount($)']

# Replace the values in 'Amount($)' with NaN where the pattern is found
df4.loc[condition, 'Amount($)'] = pd.NA

df4.groupby(by = 'Amount($)', as_index = False).sum()

In [ ]:
df4.groupby(by = 'Investor', as_index = False).sum()

In [ ]:
df4['Investor'] = df4['Investor'].fillna('')

In [ ]:
# Identify rows where 'Investor' column contains '$' symbol
condition = df4['Investor'].str.contains('1000000')

# Move values with '$' symbol from 'Investor' to 'Amount($)' column
df4.loc[condition, 'Amount($)'] = df4.loc[condition, 'Investor']

# Remove the '$' symbol from the 'Amount($)' column
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')

# Replace remaining values in 'Investor' with NaN
# df4.loc[condition, 'Investor'] = pd.NA

# print(df)

In [ ]:
# Identify rows where 'Investor' column contains '$' symbol
condition = df4['Investor'].str.contains('5000000')

# Move values with '$' symbol from 'Investor' to 'Amount($)' column
df4.loc[condition, 'Amount($)'] = df4.loc[condition, 'Investor']

# Remove the '$' symbol from the 'Amount($)' column
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')

# Replace remaining values in 'Investor' with NaN
# df4.loc[condition, 'Investor'] = pd.NA

# print(df)

In [ ]:
# Identify rows where 'Investor' column contains '$' symbol
condition = df4['Investor'].str.contains('1000000')

# Move values with '$' symbol from 'Investor' to 'Amount($)' column
df4.loc[condition, 'Amount($)'] = df4.loc[condition, 'Investor']

# Remove the '$' symbol from the 'Amount($)' column
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')

# Replace remaining values in 'Investor' with NaN
# df4.loc[condition, 'Investor'] = pd.NA

# print(df)

In [ ]:
# Identify rows where 'Investor' column contains '$' symbol
condition = df4['Investor'].str.contains('22000000')

# Move values with '$' symbol from 'Investor' to 'Amount($)' column
df4.loc[condition, 'Amount($)'] = df4.loc[condition, 'Investor']

# Remove the '$' symbol from the 'Amount($)' column
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')
df4['Investor'] = df4['Investor'].str.replace('1000000', 'NA')
df4['Investor'] = df4['Investor'].str.replace('1000000\t#REF!', 'NA')
# Replace remaining values in 'Investor' with NaN
# df4.loc[condition, 'Investor'] = pd.NA

# print(df)

In [ ]:
df4.groupby(by = 'Investor', as_index = False).sum()

In [ ]:
df4.Investor.isna().sum()

In [ ]:
df4.describe().T

In [ ]:
# Convert Amount column from object to float data type

# Remove currency symbol $
df4['Amount($)'] = df4['Amount($)'].str.replace('$', '')

# Remove the comma (,) symbol
df4['Amount($)'] = df4['Amount($)'].str.replace(',', '')
df4['Amount($)'] = df4['Amount($)'].str.replace('', '0')
# Replace all strings longer than 9 character
max_length = 9
df4['Amount($)'] = df4['Amount($)'].apply(lambda x: '0' if len(x) > max_length else x)

# Remove all white spaces
df4['Amount($)'] = df4['Amount($)'].str.strip()

df4['Amount($)'] = df4['Amount($)'].astype(np.float64).round(2)

In [ ]:
df4.isna().sum()

In [ ]:
df4.info()

In [ ]:
# Concatenate all DataFrames along rows
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# If there are overlapping columns, you can handle them separately if needed

# Print the concatenated DataFrame
df

In [ ]:
df.drop(columns=['column10'], inplace=True)
df

In [ ]:
df.describe().T.round(2)

In [ ]:
df.isna().sum()

In [ ]:
df['Amount($)'].mean().round(2)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# df.duplicated().sum()

In [ ]:
# df.drop_duplicates()

### Visualizing Characteristics of the Dataset

In [ ]:
# Visualize the distribution of the start-ups' Stage with boxplot
df.plot.box(column='Amount($)', by='Year')

In [ ]:
# Visualize the distribution of the start-ups' Location with boxplot
df.plot.box(column='Amount($)', by='Founded')

In [ ]:
# Distribution of the variables
df.hist(figsize=(20, 15))